## Additional Methods

参考 blog: [collections 模块之 deque](https://blog.csdn.net/chl183/article/details/106958004)

***methods:*** (粗体为书中未提及)

- ***.count( ), .insert(index, obj), .remove( ), .clear( )***
- *maxlen=, .rotate(n)*
- (可添加后缀left) *.append( ), .extend( ), .pop( )*

In [23]:
import collections
d = collections.deque("abcdefg1234567cba")

print( d.count('b') )

print( 'before:  {}'.format(d) )
d.insert(10,'xxx') # 在指定位置插入元素(idx左起计算)
print( 'after:   {}'.format(d) )
d.remove('a') # 移除左起第一次出现的匹配元素
print( 'removed: {}'.format(d) )

d.clear()
print( 'cleared: {}'.format(d) )


2
before:  deque(['a', 'b', 'c', 'd', 'e', 'f', 'g', '1', '2', '3', '4', '5', '6', '7', 'c', 'b', 'a'])
after:   deque(['a', 'b', 'c', 'd', 'e', 'f', 'g', '1', '2', '3', 'xxx', '4', '5', '6', '7', 'c', 'b', 'a'])
removed: deque(['b', 'c', 'd', 'e', 'f', 'g', '1', '2', '3', 'xxx', '4', '5', '6', '7', 'c', 'b', 'a'])
cleared: deque([])


可参考：[Python 官方文档 collections 模块](https://docs.python.org/zh-cn/3.9/library/collections.html#collections.namedtuple)

以获得更多内置属性

In [45]:
import collections

Point = collections.namedtuple('Point', 'x,y')
t = [11,22]
Point._make(t)

Point(x=11, y=22)

## 2.2.4 deque: Double-Ended Queue

双端队列支持从任意一端增加和删除元素。（栈和队列可视为其退化形式）

deque 是一种序列容器，因此同样支持list的一些操作（如__getitem(), len(), 通过标识符从队列中间删除元素

In [2]:
# CODE LIST 2-24
import collections

d = collections.deque('abcdefg')
print('Deque:', d)
print('Length:', len(d))
print('Left end:', d[0])
print('Right end:', d[-1])

d.remove('c')
print('remove(c):', d)
print(d.__getitem__(0))

Deque: deque(['a', 'b', 'c', 'd', 'e', 'f', 'g'])
Length: 7
Left end: a
Right end: g
remove(c): deque(['a', 'b', 'd', 'e', 'f', 'g'])
a
<built-in method append of collections.deque object at 0x00000173F00B1C18>


### 2.2.4.1 Populating / 填充

- ***.extend( ), .append( )***
- *.extendleft( ), .appendleft( )*

In [2]:
# CODE LIST 2-25
import collections

# Add to the right
d1 = collections.deque()
d1.extend('abcdefg')
print('extend    :', d1)
d1.append('h')
print('append    :', d1)

# Add to the left
d2 = collections.deque()
d2.extendleft(range(6))
print('extendleft:', d2) # 依次从左侧填充，此处相当于逆序填充
d2.appendleft(6)
print('appendleft:', d2)

extend    : deque(['a', 'b', 'c', 'd', 'e', 'f', 'g'])
append    : deque(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])
extendleft: deque([5, 4, 3, 2, 1, 0])
appendleft: deque([6, 5, 4, 3, 2, 1, 0])


### 2.2.4.2 Consuming / 消费

- ***.pop( ), .popleft( )***

双端队列是 **线程安全** 的，所以甚至可以在不同线程中同时从两端消费队列的内容。

In [9]:
# CODE LIST 2-26
import collections

print('From the right:')
d = collections.deque('abcdefg')
while True:
    try:
        print(d.pop(), end='')
    except IndexError:
        break
print()

print('\nFrom the left:')
d = collections.deque(range(6))
while True:
    try:
        print(d.popleft(), end='')
    except IndexError:
        break
print()

From the right:
gfedcba

From the left:
012345


In [13]:
# CODE LIST 2-27
import collections
import threading
import time

candle = collections.deque(range(5))


def burn(direction, nextSource):
    while True:
        try:
            next = nextSource()
        except IndexError:
            break
        else:
            print('{:>8}: {}'.format(direction, next))
            time.sleep(0.1)
    print('{:>8} done'.format(direction))
    return


left = threading.Thread(target=burn,
                        args=('Left', candle.popleft))
right = threading.Thread(target=burn,
                         args=('Right', candle.pop))

left.start()
right.start()

left.join()
right.join()


    Left: 0
   Right: 4
   Right: 3    Left: 1

    Left: 2   Right done

    Left done


### 2.2.4.3 Rotating

- ***.rotate(n)***

按任意一个方向旋转，从而跳过一些元素。$n>0$ 则从右端取元素移到左端。

旋转的法则类似于**拨号盘**或者**轮盘**。

In [18]:
# CODE LIST 2-28
import collections

d = collections.deque(range(10))
print('Normal        :', d)

d = collections.deque(range(10))
d.rotate(2)
print('Right rotation:', d)

d = collections.deque(range(10))
d.rotate(-2)
print('Left rotation :', d)

Normal        : deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Right rotation: deque([8, 9, 0, 1, 2, 3, 4, 5, 6, 7])
Left rotation : deque([2, 3, 4, 5, 6, 7, 8, 9, 0, 1])


### 2.2.4.4 Constraining the Queue Size

- ***.deque(..., maxlen = ...)*** optional para: maxlen 指定最大长度

达到最大长度后，从一端增加元素会**从另一端删除**现有元素。

In [1]:
# CODE LIST 2-29
import collections
import random

# Set the random seed so we see the same output each time
# the script is run.
random.seed(114514)

d1 = collections.deque(maxlen=3)
d2 = collections.deque(maxlen=3)

for i in range(5):
    n = random.randint(0, 100)
    print('n =', n)
    d1.append(n)
    d2.appendleft(n)
    print('D1:', d1)
    print('D2:', d2)

n = 29
D1: deque([29], maxlen=3)
D2: deque([29], maxlen=3)
n = 13
D1: deque([29, 13], maxlen=3)
D2: deque([13, 29], maxlen=3)
n = 89
D1: deque([29, 13, 89], maxlen=3)
D2: deque([89, 13, 29], maxlen=3)
n = 26
D1: deque([13, 89, 26], maxlen=3)
D2: deque([26, 89, 13], maxlen=3)
n = 20
D1: deque([89, 26, 20], maxlen=3)
D2: deque([20, 26, 89], maxlen=3)
deque([26, 89, 1], maxlen=3)


## 2.2.5 namedtuple: Tuple Subclass with Named Fields 
带命名字段的元组（tupple）子类

标准tuple用数值索引来访问成员（从0计之下标）。<br>
namedtuple 在数值索引之外还可为之指定名字。

In [24]:
# CODE LIST 2-30
bob = ('Bob', 30, 'male')
print('Representation:', bob)

jane = ('Jane', 29, 'female')
print('\nField by index:', jane[0])

print('\nFields by index:')
for p in [bob, jane]:
    print('{} is a {} year old {}'.format(*p))

Representation: ('Bob', 30, 'male')

Field by index: Jane

Fields by index:
Bob is a 30 year old male
Jane is a 29 year old female


### 2.2.5.1 定义

*.nametuple( )* **工厂函数** (*factory fuction*): 创建tuple子类，用此子类来定义各个元组。

***collections.namedtuple(typename, field_names, \*, verbose=False, rename=False, module=None)***

*args:* *typename* 所返回子类的名称；*field_names*: 子类元组中成员的命名，用空格或逗号隔开。

具有**不可修改性/immutable**

In [31]:
# CODE LIST 2-31
import collections

Person = collections.namedtuple('Person', 'name age')

bob = Person(name='Bob', age=30)
print('\nRepresentation:', bob)

jane = Person(name='Jane', age=29)
print('\nField by name:', jane.name)

print('\nFields by index:')
for p in [bob, jane]:
    print('{} is {} years old'.format(*p)) 
    # p是元组子类，传入打印内容时用*号
    # *: 向函数传递参数，将变量中可迭代对象的元素拆解出来，作为独立的参数传递给函数


Representation: Person(name='Bob', age=30)

Field by name: Jane

Fields by index:
Person(name='Bob', age=30)
Bob is 30 years old
Person(name='Jane', age=29)
Jane is 29 years old


In [34]:
# CODE LIST 2-32
import collections

Person = collections.namedtuple('Person', 'name age')

pat = Person(name='Pat', age=12)
print('\nRepresentation:', pat)

try: 
    pat.age = 21
except:
    print("AttributeError: can't set attribute")


Representation: Person(name='Pat', age=12)
AttributeError: can't set attribute


### 2.2.5.2 Invalid Field Names / 非法字段名

非法：字段名重复或与Python关键字冲突，此时解析字段名会产生 ValueError 异常

设置工厂函数的 option: *rename=True* 。无效字段名 field_names  会自动转换成**位置名**。

In [35]:
# CODE LIST 2-33
import collections

try:
    collections.namedtuple('Person', 'name class age')
except ValueError as err:
    print(err)

try:
    collections.namedtuple('Person', 'name age age')
except ValueError as err:
    print(err)

Type names and field names cannot be a keyword: 'class'
Encountered duplicate field name: 'age'


In [36]:
# CODE LIST 2-34
import collections

with_class = collections.namedtuple(
    'Person', 'name class age',
    rename=True)
print(with_class._fields)

two_ages = collections.namedtuple(
    'Person', 'name age age',
    rename=True)
print(two_ages._fields)

('name', '_1', 'age')
('name', 'age', '_2')


### 2.2.5.3 Special Attributes

**内置属性名**都有一个下划线(_)前缀。可用于处理子类和实例

- ***._make(iterable), ._asdict( ), _replace(\*\*kwargs), .fields***

In [40]:
# CODE LIST 2-35
# .field : 子类的字段名
import collections

Person = collections.namedtuple('Person', 'name age')

bob = Person(name='Bob', age=30)
print('Representation:', bob)
print('Fields:', bob._fields)

Representation: Person(name='Bob', age=30)
Fields: ('name', 'age')


In [41]:
# CODE LIST 2-36
# ._asdict() : 将实例转换为 OrderedDict 实例
import collections

Person = collections.namedtuple('Person', 'name age')

bob = Person(name='Bob', age=30)
print('Representation:', bob)
print('As Dictionary:', bob._asdict())

Representation: Person(name='Bob', age=30)
As Dictionary: OrderedDict([('name', 'Bob'), ('age', 30)])


In [39]:
# CODE LIST 2-37
# ._replace() 返回一个《新的》命名元组实例，并替换指定域的值
import collections

Person = collections.namedtuple('Person', 'name age')

bob = Person(name='Bob', age=30)
print('\nBefore:', bob)
bob2 = bob._replace(name='Robert')
print('After:', bob2)
print('Same?:', bob is bob2)


Before: Person(name='Bob', age=30)
After: Person(name='Robert', age=30)
Same?: False


## 2.2.6 OrderedDict: Remember the Order Keys Are Added to a Dictionary
可以记住内容增加顺序的字典（dict）子类

常规 dict 在迭代处理中根据散列表中储存键的顺序生成值，这会受到一个随机值的影响<br>
而 OrderedDict 会记住元素插入的顺序并用于创建迭代器。

In [54]:
# CODE LIST 2-38
import collections

print('Regular dictionary:')
d = {}
d['a'] = 'A'
d['b'] = 'B'
d['c'] = 'C'

for k, v in d.items():
    print(k, v, end=' -- ')

print('\nOrderedDict:')
d = collections.OrderedDict()
d['a'] = 'A'
d['b'] = 'B'
d['c'] = 'C'

for k, v in d.items():
    print(k, v, end=' -- ')

Regular dictionary:
a A -- b B -- c C -- 
OrderedDict:
a A -- b B -- c C -- 

### 2.2.6.1 Equality

有序字典的相等还包含了插入顺序。

In [55]:
# CODE LIST 2-39
import collections

print('dict       :', end=' ')
d1 = {}
d1['a'] = 'A'
d1['b'] = 'B'
d1['c'] = 'C'

d2 = {}
d2['c'] = 'C'
d2['b'] = 'B'
d2['a'] = 'A'

print(d1 == d2)

print('OrderedDict:', end=' ')

d1 = collections.OrderedDict()
d1['a'] = 'A'
d1['b'] = 'B'
d1['c'] = 'C'

d2 = collections.OrderedDict()
d2['c'] = 'C'
d2['b'] = 'B'
d2['a'] = 'A'

print(d1 == d2)

dict       : True
OrderedDict: False


### 2.2.6.2 Reordering

***.move_to_end(key, last=True)***

将 key 移动到有序字典的一端，default last=True -> 末尾; last=False -> 开头。

In [56]:
# CODE LIST 2-40
import collections

d = collections.OrderedDict(
    [('a', 'A'), ('b', 'B'), ('c', 'C')]
)

print('Before:')
for k, v in d.items():
    print(k, v)

d.move_to_end('b')

print('\nmove_to_end():')
for k, v in d.items():
    print(k, v)

d.move_to_end('b', last=False)

print('\nmove_to_end(last=False):')
for k, v in d.items():
    print(k, v)

Before:
a A
b B
c C

move_to_end():
a A
c C
b B

move_to_end(last=False):
b B
a A
c C
